In [91]:
from synonymscrawler import simple_synonyms_crawler
from multiprocessing.dummy import Pool as ThreadPool

In [92]:
import os
import numpy as np
from gensim import models
import string
from keras.preprocessing.text import Tokenizer
from tqdm import tqdm
import math

In [93]:
BASE_DIR = '../'
EMBEDDING_DIR = BASE_DIR + 'alpha/embeddings/'  # http://nlp.stanford.edu/projects/glove/ pretrained vectors
EMBEDDING_FILE = "GoogleNews-vectors-negative300.bin"
TEXT_DATA_DIR = BASE_DIR + 'data/'
TEXT_DATA_FILE = "movie_reviews.csv"
HEADER = True

In [94]:
def load_data():
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split(",", 1)
            x.append(temp_x)
            y.append(temp_y)

    return x, y

In [95]:
X, y = load_data()
y = np.array(y, dtype='int8')

In [96]:
tokenizer = Tokenizer(filters='"«#$%&()*+-/:;<=>@[\\]^{|}~\t\n,.!?_[1-9]')
tokenizer.fit_on_texts(X)

In [105]:
def writeDict(d, filename, sep=";"):
    with open(filename, "a") as f:
        for i in d.keys():            
            f.write(i + sep + sep.join([str(x) for x in d[i]]) + "\n")
            
def readDict(filename, sep=";"):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.rstrip().split(sep)
            dict[values[0]] = values[1:]
        return(dict)

In [98]:
def get_synonyms(x):
    try:
        return simple_synonyms_crawler.crawl(x, 3)
    except:
        return [x, x, x]

In [100]:
arr = list(range(0,len(tokenizer.word_index.keys()),10000)) + [len(tokenizer.word_index.keys())]
for n in range(len(arr)-1):
    print(n)
    d = {}
    pool = ThreadPool(16)
    results = pool.map(lambda w: get_synonyms(w), list(tokenizer.word_index.keys())[arr[n]:arr[n+1]])
    for i in results:
        d[i[0]] = i[1:]
    writeDict(d, "synonyms.txt")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [86]:
len(arr)-1

15

In [106]:
d = readDict("synonyms.txt")

In [107]:
d["good"]

['great', 'satisfying']